# Dom's Notebook

hypothesis:

(01/09/20)
- Taxo ID [7,8] fall into Exec and consult
- taxo ID [3,4,5,9] fall into specialists
- Taxo ID [1,2,6] are further divided by experience level
        - we weren't satisfied on the results of this grouping

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../')
import wrangle, explore
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import minmax_scale, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import ttest_ind, chi2_contingency


ModuleNotFoundError: No module named 'wrangle'

In [ ]:
data = pd.read_excel('/Users/dom/codeup_data_science/capstone/kev/survey_responses.xlsx', )
data.head(2)

In [ ]:
data_dictionary = pd.read_excel('../data_files/data_dictionary.xlsx')
data_dictionary.head()

In [ ]:
data.rename(columns = data_dictionary[['qid','column_name']].set_index('qid').column_name, inplace=True)
data.head(2)

In [ ]:
df,_ = wrangle.wrangle_data()
df.head()

- Checking freds nlp functions

In [ ]:
# sys.path.append('../fred')
# import nlp

In [ ]:
# mat, vect = nlp.create_tfidf_matrix(df.best_event)

#### Exploration

In [ ]:
#dataframe
df.head(2)

In [ ]:
#Quantitative columns
quant_cols = df.select_dtypes([int,float]).columns
quant_cols.to_list()

In [ ]:
#Jobs by taxo id
    #1 is UX researchers
    #2 is Developers
    #3 is Researcher/Scientist
    #4 is Academic/Students
    #5 is Planners?
    #6 is Design/Research Managers
    #7 is Consultans
    #8 is Executive
    #9 is misc
data.groupby(['job_taxo', 'job_title']).job_title.count()

In [ ]:
df.groupby(['persona_id','job_taxo']).persona_id.count()

In [ ]:
df.persona_id.value_counts()

In [ ]:
exp_cols = ['exp_conduct_res', 'exp_analyze_res',
           'exp_buy_res_report', 'exp_manage_res_proj', 'exp_observe_res',
           'exp_plan_res', 'exp_teach_res', 'exp_advocate_res',
           'exp_hire_res_vendor', 'exp_lead_res_team']
df[exp_cols].mean(axis = 1)

In [ ]:
#show distributions of values


In [ ]:
#establish features and target and split train and test data
def assign_target(x):
    group=0
    if x.learning_conference > 1:
        group += 1
    if x.likely_conference > 1:
        group += 1
    return group
df['target'] = df.apply(assign_target, axis = 1)

x = quant_cols.drop(['likely_conference','learning_conference', 'likely_workshop','learning_workshop']).to_list()+['persona_id', 'job_taxo']
y = 'target'
train, test = train_test_split(df[x+[y]].dropna(subset = [y]), stratify = df[y])

In [ ]:
#1/14/20
#use svm or 
#form hypothesis and test them
df.target.value_counts()

In [ ]:
plt.figure(figsize=(14,11))
sns.heatmap(df.corr(), cmap = 'icefire_r')
plt.show()

In [ ]:
df.groupby('persona_id').likely_conference.agg(['mean', 'median',]).plot.bar()
plt.show()
df.groupby('persona_id').likely_conference.count()

In [ ]:
df.groupby('persona_id').learning_conference.agg(['mean', 'median']).plot.bar()
plt.show()


In [ ]:
df.groupby('persona_id').target.agg(['mean', 'median']).plot.bar()
plt.ylim(.8, 2.2)
plt.show()
df.groupby('persona_id').target.count()


In [ ]:
ttests = {}
row = {}
for i in df.persona_id.unique().sort_values(ascending = False):
    for j in df.persona_id.unique().sort_values(ascending = False):
        ttests[j] = ttest_ind(df[df.persona_id == i].target,df[(df.persona_id == j)].target, nan_policy='omit')[1]
    row[i] = ttests.copy()
sns.heatmap(pd.DataFrame(row), vmin = .01, vmax=.05)
plt.title('Statistical difference between persona groups of likelyhood to attend a conference')

In [ ]:
pd.DataFrame(row)

After consideration, I have decided to scrap this model and opt to use chi squared tests to determine which features to eliminate

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, max_features=20, random_state=123)
tree.fit(train[x], train[y])
predicted = tree.predict(test[x]) 


In [ ]:
print(classification_report(test[y], predicted))
pd.crosstab(test[y], predicted)

In [ ]:
plot_tree(tree, feature_names=train[x].columns)

In [ ]:
pd.DataFrame(tree.feature_importances_, index = train[x].columns)

In [ ]:
reg = SVC(random_state=123)
reg.fit(train[x], train[y])
predicted = reg.predict(test[x])
r2 = reg.score(test[x], test[y])

In [ ]:
print(classification_report(test[y], predicted))
pd.crosstab(test[y], predicted)

# MVP Model

In [ ]:
df_mvp = df[['job_taxo', 'persona_id', 'num_employees']].join(pd.DataFrame(df[['exp_conduct_res']].apply(max, axis = 1), columns = ['experience']),).dropna()

In [ ]:
#one hot encode for job taxos
df_mvp_ohe = pd.get_dummies(df_mvp, columns=['job_taxo']) 

In [ ]:
x = df_mvp_ohe.drop(columns=['persona_id']).columns.to_list()
y = 'persona_id'
train, test = train_test_split(df_mvp_ohe, random_state = 123, stratify = df_mvp_ohe.persona_id)

In [ ]:
classifier = SVC(random_state=123)
classifier.fit(train[x], train[y])
predicted = classifier.predict(test[x])

In [ ]:
print(classification_report(test[y], predicted))
pd.crosstab(test[y], predicted)

In [ ]:
tree = DecisionTreeClassifier(random_state=123)
tree.fit(train[x], train[y])
predicted = tree.predict(test[x]) 

In [ ]:
print(classification_report(test[y], predicted))
pd.crosstab(test[y], predicted)

### Explore groups

In [ ]:
df.groupby(['persona_id','job_title']).job_taxo.count()

In [ ]:
df.groupby(['persona_id','num_employees'], ).job_taxo.count()

In [ ]:
df.groupby(['persona_id','session_network']).job_taxo.count()

In [ ]:
df.groupby(['persona_id','factor_speaker']).job_taxo.count()

In [ ]:
df.groupby(['persona_id','factor_diverse_speak']).job_title.count()

In [ ]:
df.groupby('persona_id').job_title.count()

In [ ]:
sns.boxplot(x = df.persona_id, y= df.target)

In [ ]:
df.columns

In [ ]:
df.num_employees

In [ ]:
for i in quant_cols:
    df[i].plot.hist()
    plt.title(i)
    plt.show()

In [ ]:
df.persona_id.unique()

In [ ]:
df.target.value_counts()

In [ ]:
df.persona_id.value_counts()/df.persona_id.count()

In [ ]:
def pct_trained_in_research(df, persona_column = 'persona_id'):
    pass
    

In [ ]:
chi2 = pd.read_csv('../kev/chi2_2dfall.txt', index_col='index')
sns.heatmap(pd.crosstab(index = chi2.chk_col, columns=chi2.vs_col,values = chi2.p_val, aggfunc='median'))

In [ ]:
chi2_row = {}
chi2_df = {}
for i in quant_cols:
    for j in quant_cols:
        observations = pd.crosstab(index = df[i], columns = df[j])
        chi2_row[j] = chi2_contingency(observations)[1]
    chi2_df[i] = chi2_row.copy()
chi2_df = pd.DataFrame(chi2_df)

In [ ]:
plt.figure(figsize=(14,14))
sns.heatmap(chi2_df,cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('big bad mega chi2 plot muah hah hah hah')
plt.show()

In [ ]:
job_qs  = ['job_conduct_res', 'job_analyze_res', 'job_buy_res_report',
           'job_manage_res_proj', 'job_observe_res', 'job_plan_res',
           'job_teach_res', 'job_advocate_res', 'job_hire_res_vendor',
           'job_lead_res_team',]
sns.heatmap(chi2_df.loc[job_qs,job_qs],cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('How much is your current job devoted to...')

In [ ]:
exp_qs  = ['exp_conduct_res', 'exp_analyze_res',
           'exp_buy_res_report', 'exp_manage_res_proj', 'exp_observe_res',
           'exp_plan_res', 'exp_teach_res', 'exp_advocate_res',
           'exp_hire_res_vendor', 'exp_lead_res_team',]
sns.heatmap(chi2_df.loc[exp_qs,exp_qs],cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('How much experience do you have with...')

In [ ]:
learning_qs = ['learning_talks',
               'learning_read', 'learning_meetup', 'learning_workshop',
               'learning_conference']
sns.heatmap(chi2_df.loc[learning_qs,learning_qs],cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('How much experience do you have with...')

In [ ]:
likely_qs  =  ['likely_watch_video', 'likely_internet',
               'likely_book', 'likely_online_group', 'likely_colleague',
               'likely_meetup', 'likely_conference', 'likely_workshop',]
sns.heatmap(chi2_df.loc[likely_qs,likely_qs],cmap = 'BuGn', vmax=.1, vmin=.01)
plt.title('How much experience do you have with...')

In [ ]:
factors = ['factor_speaker', 'factor_diverse_speak', 'factor_topics', 
           'factor_format_sessions', 'factor_size', 'factor_network', 
           'factor_variety_attend', 'factor_code', 'factor_location', 
           'factor_ability_to_pay']
sns.heatmap(chi2_df.loc[factors,factors],cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('factor questions')

In [ ]:
session_qs  = ['session_keynote', 'session_long_talk',
               'session_short_talk', 'session_workshop', 'session_network',
               'session_social_event', 'session_qa', 'sesson_topic_tables']
sns.heatmap(chi2_df.loc[session_qs,session_qs],cmap = 'BuGn', vmax=.05, vmin=.01)
plt.title('most important session types')

In [ ]:
for i in df.persona_id.unique():
    print(df[df.persona_id == i].factor_speaker.value_counts().sort_index(),f'[{i}]')

In [ ]:
def encode_factor_speaker(x):
    if x == 0:
        return 0
    elif x == 1 or x == 2:
        return 1
    elif x == 3:
        return 2
    elif x == 4:
        return 3
    return x

In [ ]:
for i in df.persona_id.unique():
    print(f'[ {i} ]')
    print(df[df.persona_id == i].factor_diverse_speak.apply(encode_factor_speaker).value_counts().sort_index(),)

In [ ]:
pd.crosstab(df.persona_id, df.factor_speaker,).plot(kind = 'bar', stacked = True, cmap = 'Greys').set_facecolor('xkcd:salmon')


In [ ]:
pd.crosstab(df.persona_id, df.factor_diverse_speak.apply(encode_factor_speaker),).plot(kind = 'bar', stacked = True, cmap = 'Greys').set_facecolor('xkcd:salmon')

In [ ]:
matrix = pd.crosstab(df.factor_network.apply(encode_factor_speaker),df.persona_id, )
# .plot(kind = 'bar', stacked = True, cmap = 'Greys').set_facecolor('xkcd:salmon')
for i in matrix:
    matrix[i] = matrix[i]/sum(matrix[i])
matrix

In [ ]:
def create_stacked_bars(df, factor, return_matrix = False):
    matrix = pd.crosstab(df[factor].apply(encode_factor_speaker),df.persona_id,)
    for i in matrix:
        matrix[i] = matrix[i]/sum(matrix[i])
    ax = matrix.transpose().plot(kind = 'barh', stacked = True, cmap = 'Greys')
    ax.set_facecolor('xkcd:light grey')
    plt.title(factor)
    plt.legend().remove()
    plt.ylabel(' ')
    plt.xticks([],[])
    plt.yticks([0,1,2,3,4],['Exec & Consult', 'Specialist','High Experience','Mid Experience','Low Experience'])
    plt.show()
    if return_matrix is True:
        return matrix


In [ ]:
matricies = {}
j = 0
for i in ['factor_speaker','factor_diverse_speak','factor_network']:
    matricies[j] = create_stacked_bars(df, i, return_matrix=True)
    j += 1


In [ ]:
persona = {}
all_personas = {}
titles = ['Execs/Consult', 'Specialists', 'Very Expd', 'Med Expd', 'Less Expd']
for j in range(1,6):
    for i in [0,1,2]:
        persona[matricies[i].index.name] = matricies[i][j]
    all_personas[j] = pd.DataFrame(persona.copy())
for dataframe in all_personas.keys():
    ax = all_personas[dataframe].transpose().plot(kind = 'barh', stacked = True, cmap = 'Blues')
    ax.set_facecolor('xkcd:light grey')
    plt.title(dataframe)
    plt.legend().remove()
    plt.ylabel(' ')
    plt.xticks([],[])
    plt.annotate('Networking Opportunities', (0,2.4), fontsize=15)
    plt.annotate('Diversity of Speakers', (0,1.4), fontsize=15)
    plt.annotate('Speaker Name Recognition',(0,.4),fontsize=15)
    plt.annotate('',xy=(1 , -.5), xycoords='data',
                    xytext=(0, -.5), textcoords='data',
                    arrowprops=dict(arrowstyle="->",
                    connectionstyle="arc3"))
    plt.yticks([],[])
    plt.ylim(-.7,2.8)
    plt.xlim(-.1,1.1)
    sns.despine(left=True, bottom=True, right=True)

    plt.savefig(f'/Users/dom/codeup_data_science/capstone/viz_files/their_personas/{dataframe}_stacked_bars', transparent = True)

In [ ]:
idx = pd.IndexSlice
persona_counts = (pd.DataFrame(df
                               .groupby(by=['persona_id'])
                               .target
                               .agg('count')
                              )
                  .rename(columns={'target': 'count'})
                 )
job_group_by = ['persona_id', 'job_title']
job_counts = (pd.DataFrame(df
                           .groupby(by=job_group_by)
                           .target
                           .agg('count')
                          )
              .reset_index()
              .rename(columns={'target': 'count'})
              .sort_values(by=['persona_id','count'], ascending=[True,False]))
job_counts = job_counts.merge(
    persona_counts.rename(columns={'count':'total'}), 
    how='left', left_on='persona_id', right_on='persona_id')
job_counts['pid_pct'] = np.array(job_counts['count'])/np.array(job_counts['total'])
job_counts = job_counts.drop(columns='total')
job_counts = job_counts.set_index(['persona_id', 'job_title'])
for p in persona_counts.index:
    jc = job_counts.loc[idx[[p],:]].head(3).copy()
    display(jc)

In [ ]:
jc.pid_pct.max()*1.1

In [ ]:
titles = ['Execs_Consult', 'Specialists', 'Very_Expd', 'Med_Expd', 'Less_Expd']

# Consultant Job Titles
print(titles[0])
jc = job_counts.loc[idx[[1],:]].head(3).copy()
jc=jc.reset_index().sort_values(by='pid_pct')
fig, ax1 = plt.subplots(figsize=(6, 4))
ax1.barh(jc.job_title, width=jc.pid_pct, height=.5, tick_label = jc.job_title, color = '#00297f')
ax1.set_facecolor('xkcd:light grey')
pcts = [int(round(jc["pid_pct"][0]*100)), int(round(jc["pid_pct"][1]*100))]
plt.annotate(f'Consultant {pcts[0]}%', (0,1.3), fontsize = 15)
plt.annotate(f'Executive {pcts[1]}%'  ,(0,.3), fontsize = 15)
plt.ylim(-.5,2.1)
plt.xlim(-.1*jc.pid_pct.max(), jc.pid_pct.max()*1.1)
plt.yticks([],[])
plt.xticks([],[])
sns.despine(left=True, bottom=True, right=True)

plt.savefig(f'/Users/dom/codeup_data_science/capstone/viz_files/their_personas/{titles[0]}_titles.png', transparent = True)

# Everyone Else Job Titles
for i in range(2,6):
    print(titles[i-1])
    persona_titles = []
    jc = job_counts.loc[idx[[i],:]].head(3).copy()
    jc = jc.reset_index().sort_values(by='pid_pct')
    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax1.barh(jc.job_title, width=jc.pid_pct, height=.5, tick_label = jc.job_title, color = '#00297f')
    ax1.set_facecolor('xkcd:light grey')
    pcts = [int(round(jc["pid_pct"][0]*100)), int(round(jc["pid_pct"][1]*100)),int(round(jc["pid_pct"][2]*100))]
    plt.annotate(f'{jc.job_title[0]} {pcts[0]}%', (0,2.3), fontsize = 15)
    plt.annotate(f'{jc.job_title[1]} {pcts[1]}%', (0,1.3), fontsize = 15)
    plt.annotate(f'{jc.job_title[2]} {pcts[2]}%'  ,(0,.3), fontsize = 15)
    plt.yticks([],[])
    plt.xticks([],[])
    plt.ylim(-.7,2.8)
    plt.xlim(-.1*jc.pid_pct.max(), jc.pid_pct.max()*1.1)
    sns.despine(left=True, bottom=True, right=True)

    plt.savefig(f'/Users/dom/codeup_data_science/capstone/viz_files/their_personas/{titles[i-1]}_titles.png', transparent = True)

In [ ]:
#lets bust out these pi charts hoe
def sm_med_large_co(x):
    if x is 0 or x is 1 or x is 2:
        return 1
    if x is 3 or x is 4:
        return 2


In [ ]:
matricies = {}
co_size_matrix = pd.crosstab(df.persona_id,df.num_employees.apply(int).apply(sm_med_large_co))
matricies[0] = co_size_matrix.transpose()
res_edu_matrix = pd.crosstab(df.persona_id,df.research_educ_cat).rename(columns = {0:1,1:2})
matricies[1] = res_edu_matrix.transpose().sort_index()
matricies

In [ ]:
persona = {}
all_personas = {}
titles = ['Execs/Consult', 'Specialists', 'Very Expd', 'Med Expd', 'Less Expd']
for j in range(1,6):
    for i in [0,1]:
        persona[matricies[i].index.name] = matricies[i][j]/sum(matricies[i][j])
    all_personas[j] = pd.DataFrame(persona.copy())
for dataframe in all_personas.keys():
    ax = all_personas[dataframe].transpose().plot(kind = 'barh', stacked = True, cmap = 'Blues')
    ax.set_facecolor('xkcd:light grey')
    plt.title(dataframe)
    plt.legend().remove()
    plt.ylabel(' ')
    plt.xticks([],[])
    plt.annotate('Academically Trained in Research', (0,1.4), fontsize=15)
    plt.annotate('Organization Size (>500)',(0,.4),fontsize=15)
    plt.annotate('No',  (0.05, .96))
    plt.annotate('Yes', (.885, .96), color = 'white')
    plt.annotate('No', (0.05,-.04))
    plt.annotate('Yes', (.885,-.04), color = 'white')
    plt.yticks([],[])
    plt.ylim(-.7,1.8)
    plt.xlim(-.1,1.1)
    sns.despine(left=True, bottom=True, right=True)

    plt.savefig(f'/Users/dom/codeup_data_science/capstone/viz_files/their_personas/{dataframe}_not_pie_charts', transparent = True)